![Py4Eng](img/logo.png)

# Calling C functions from Python with `ctypes`
## Yoav Ram

# `ctypes`

[ctypes](https://docs.python.org/3/library/ctypes.html) is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

In [1]:
import ctypes

We'll start by loading the standard C library (`msvcrt` on Windows, `libc` on Linux):

In [7]:
import sys
print(sys.platform)
if sys.platform == 'win32': # windows
    libc = ctypes.cdll.msvcrt
elif sys.platform == 'darwin': # osx
    libc = ctypes.CDLL('/usr/lib/libc.dylib')
else: # linux
    libc = ctypes.CDLL("libc.so.6")

darwin


(In general we can find the shared library path using `ctypes.util.find_library("c"))`).

We can now get functions from the standard library:

In [8]:
libc.printf(b"Hello, %s\n", b"World!")
libc.printf(b"%d bottles of beer\n", 42)

19

`printf` prints to the real standard output channel, not to `sys.stdout`; look for the output in the notebook server console. 

> `None`, integers, bytes objects and (unicode) strings are the only native Python objects that can directly be used as parameters in these function calls. `None` is passed as a C NULL pointer, bytes objects and strings are passed as pointer to the memory block that contains their data (char * or wchar_t *). Python integers are passed as the platforms default C int type, their value is masked to fit into the C type.

However, `printf` expects ANSI and not unicode, so we need to pass it `bytes` rather than `str`:

In [9]:
libc.printf("Hello, %s\n", "World!")

1

Another example is the `time` function:

In [10]:
print(libc.time)
libc.time(None)

1467612136

Compare with Python's `time.time`:

In [11]:
import time
time.time()

1467612138.192701

Python's `time` gave us extra precision!

Here is another example, it uses the [`strchr`](http://www.cplusplus.com/reference/cstring/strchr/) function, which expects a string pointer and a char, and returns a pointer to a string that starts with that char:

In [12]:
libc.strchr(b"abcdef", ord("d")) 

69446147

By default functions are assumed to return the C `int` type. Other return types can be specified by setting the `restype` attribute of the function object. Since `strchr` returns a string, we need to set the result type to `c_char_p` - 

In [13]:
libc.strchr.restype = ctypes.c_char_p
libc.strchr(b"abcdef", ord("d")) 

b'def'

Similarly, functions expect integers, bytes, strings and `None`, so if we want to avoid the `ord("x")` call we can set the `argtypes` attribute, and the second argument will be converted from a single character Python object into a C wchar:

In [14]:
libc.strchr.argtypes = [ctypes.c_char_p, ctypes.c_wchar]
libc.strchr(b"abcdef", "d")

b'def'

Other ctypes types can be found in the [documentation](https://docs.python.org/3/library/ctypes.html#fundamental-data-types).

## Using your own C extension

We'll use a C extension that implements the Fibonacci series using recursion.

This example follows the [SciPy Cookbook](https://scipy-cookbook.readthedocs.org/items/Ctypes.html#fibonacci-example-using-numpy-arrays-c-and-scons).

First, let's change to the example directory:

In [15]:
%cd ../scripts/ctypes/

/Users/yoavram/Work/Py4Eng/scripts/ctypes


Have a look at the C file. Note that we export the three functions (`fib`, `fibseries`, and `fibmatrix`) with `dllexport`, and that we don't use an `h`.

In [16]:
%less fibonacci.c

There's nothing special here, and the same code could be written in Python, too, with some changes:

In [17]:
import numpy as np

def fib(a):
    if a <= 0: #  Error -- wrong input will return -1. 
        return -1
    elif a == 1:
        return 0;
    elif a == 2 or a == 3:
        return 1
    else:
        return fib(a - 2) + fib(a - 1)

def fibseries(a):
    series = np.empty_like(a, dtype=int)
    for i in range(len(a)):
        series[i] = fib(a[i])
    return series

def fibmatrix(a):
    rows, columns = a.shape
    matrix = np.empty_like(a, dtype=int)
    for i in range(rows):
        for j in range(columns):
            matrix[i,j] = fib(a[i,j])
    return matrix

We need to compile `fibonacci.c`. 

- On Windows: run the `build.bat` file; but first make sure `vcvarsall.bat` is in the `PATH` environment variable.
- On Linux/OSX, you can use [scons](http://www.scons.org), which only works with Python 2.7. On *Binder* just use the system `pip` to install it (`pip install scons`); on your own machine you should create a virtual enviornment using `conda create -n py27 python=2.7 scons`, then `conda run -n activate py27` and then `scons`.

In [ ]:
!build.bat

Now that we have our shared library, let's have a look at the Python wrapper around `fibonacci.dll` (Windows) or `libfibonacci.so` (Linux):

In [23]:
%less fibonacci.py

This Python module loads the DLL, reads the relevant functions, annotates the argument and return value types, and then defines wrapper Python function that create the output arguments and communicate with the ctypes functions. There is also a lot of add docstrings. Note that we also make use of `numpy.ctypeslib`, which allows to use NumPy arrays wherever data buffers are used in the C code.

In [25]:
import fibonacci as fb

In [26]:
fb.fibseries([1, 5, 9])

array([ 0,  3, 21], dtype=int32)

Compare to the pure-Python function:

In [27]:
fibseries([1, 5, 9])

array([ 0,  3, 21])

In [28]:
%timeit fib(10)
%timeit fb.fib(10)

100000 loops, best of 3: 18.4 µs per loop
The slowest run took 13.10 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.2 µs per loop


In [29]:
v = np.arange(1, 15)
%timeit fibseries(v)
%timeit fb.fibseries(v)

1000 loops, best of 3: 955 µs per loop
The slowest run took 7.16 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 21.3 µs per loop


In [30]:
v = np.random.randint(5, 15, (5, 5))
%timeit fibmatrix(v)
%timeit fb.fibmatrix(v)

100 loops, best of 3: 2.53 ms per loop
10000 loops, best of 3: 22.5 µs per loop


It's clear that the **C implementation is significantly faster**, especially when given an array of inputs, despite the fast that it perfoms the exact same algorithm as the pure Python version does.

# References

- [ctypes](https://docs.python.org/3/library/ctypes.html)
- [SciPy Cookbook](https://scipy-cookbook.readthedocs.org/items/Ctypes.html)
- [IPython Cookbook](https://ipython-books.github.io/cookbook/)'s chapter 5 has a section about using C in Python, and some examples are also provided in a [github repo](http://nbviewer.jupyter.org/github/ipython-books/cookbook-code/blob/master/notebooks/chapter05_hpc/03_ctypes.ipynb).

## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com) and is part of the [_Python for Engineers_](https://github.com/yoavram/Py4Eng) course.

The notebook was written using [Python](http://python.org/) 3.6.1.
Dependencies listed in [environment.yml](../environment.yml), full versions in [environment_full.yml](../environment_full.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)